# Recommending Music using the AudioScrobbler Dataset

In [1]:
val uadatapath="hdfs:///user/swethakolalapudi/audio/user_artist_data.txt"
val rawUserArtistData = sc.textFile(uadatapath)

In [9]:
rawUserArtistData.map(x => x.split(" ")(2).toDouble).stats()

(count: 24296858, mean: 15.295762, stdev: 153.915321, max: 439771.000000, min: 1.000000)

In [2]:
import org.apache.spark.mllib.recommendation._

In [4]:
val uaData=rawUserArtistData.map(_.split(" ")).filter(_(2).toInt>=20).map(x => Rating(x(0).toInt,x(1).toInt,x(2).toInt))
uaData.persist()

MapPartitionsRDD[5] at map at <console>:27

In [6]:
val model=ALS.trainImplicit(uaData,10,5,0.01,1)

In [14]:
var user: Int=1000002
var recommendations=model.recommendProducts(user,5)

In [8]:
recommendations

Array(Rating(1000002,1270,1.185725442054678), Rating(1000002,1000188,1.1677245029656391), Rating(1000002,1205,1.1586795057541834), Rating(1000002,1428,1.1576052790420486), Rating(1000002,82,1.1389863112439214))

In [24]:
val artistsPath="hdfs:///user/swethakolalapudi/audio/artist_data.txt"
val artistLookup=sc.textFile(artistsPath).map(_.split("\t")).filter(_.length==2).map(x => (x(0),x(1)))
artistLookup.persist()

MapPartitionsRDD[167] at map at <console>:25

In [19]:
val userArtists=rawUserArtistData.map(_.split(" ")).filter{case Array(userId,_,rating) => (userId.toInt == user) && (rating.toInt>50)}.map(_(1)).collect()

In [27]:
for (artist <- userArtists){
     println( artistLookup.lookup(artist)(0))}

Portishead
A Perfect Circle
Aerosmith
Judas Priest
Metallica
Foo Fighters
Counting Crows
Creed
Audioslave
Muse
(hed) Planet Earth
Dire Straits
Free
Fun Lovin' Criminals
Guns N' Roses
Satriani, Joe
A
Joe Satriani
Bruce Springsteen
Goo Goo Dolls
Fugees
Michael Jackson
Roachford
Barenaked Ladies
Buckcherry
Jools Holland
The Classic Chill Out Album
Frankie Goes To Hollywood
King's X
Mr. Big
Dave Weckl
Dan Reed Network
Liquid Tension Experiment
Level 42
Rage Against the Machine
Badly Drawn Boy
Beth Orton
Dido
Lenny Kravitz
Everclear
Feeder
Jimi Hendrix
Red Hot Chili Peppers
R.E.M.
Desert Sessions
The Kleptones
Jamiroquai
Led Zeppelin
Marcus Miller
Moby
Miles Davis
Electric Wizard
Matchbox Twenty
The Police
Nina Simone
Jeff Buckley
Dream Theater
Eels
Nickelback
Diana Krall
The Jimi Hendrix Experience
Pink
Rammstein
Norah Jones
Ben Folds Five
Radiohead


In [28]:
for (rating <- recommendations){
     println( artistLookup.lookup(rating.product.toString)(0))}

Queen
Dire Straits
U2
Eric Clapton
Pink Floyd
